# 💖Fun💖Gut💖

# 00 Packages and Directory 📦

Before we do anything, we need to make sure we have all the tools we need to do analyse our data and that we have a system put in place to organize our inputs and outputs. Here we first install geopandas which we use to look at our metadata later on. Subsequently we improt all required plugins, set our data directory and create a simple numbered directory to organize our outputs.

In [1]:
pip install geopandas 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import geopandas as gpd

In [2]:
import os
import IPython
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  

import qiime2 as q2
from qiime2 import Visualization

data_dir = 'data'
PATH = "data/fungut_metadata.tsv"
surveys_df = pd.read_csv(PATH, sep="\t")

%matplotlib inline

## 00.01 File directory

In [4]:
"""
! mkdir -p "data/01"
! mkdir -p "data/02"
! mkdir -p "data/03"
! mkdir -p "data/03/denoising_4"
! mkdir -p "data/04"
! mkdir -p "data/05"
! mkdir -p "data/06"
! mkdir -p "data/07"
! mkdir -p "data/08"
"""

'\n! mkdir -p "data/01"\n! mkdir -p "data/02"\n! mkdir -p "data/03"\n! mkdir -p "data/04"\n! mkdir -p "data/05"\n! mkdir -p "data/06"\n! mkdir -p "data/07"\n! mkdir -p "data/08"\n'

# 01 Data import 📥

## 01.01

Here we imoprt our data and blablabla

In [2]:
!wget https://polybox.ethz.ch/index.php/s/bLQ6eRWEZo8KmFo/download -O $data_dir/fungut_metadata.tsv

--2025-11-20 21:16:09--  https://polybox.ethz.ch/index.php/s/cXXGDT4rkYT8ADg/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘data/metadata_original.tsv’

data/metadata_origi     [ <=>                ]  18.65K  --.-KB/s    in 0.005s  

2025-11-20 21:16:09 (3.69 MB/s) - ‘data/metadata_original.tsv’ saved [19102]



In [5]:
! qiime tools peek $data_dir/fungut_forward_reads.qza

UUID:        3638611d-1767-413b-9390-70ee3d78e4ff
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [2]:
! qiime demux summarize \
    --i-data $data_dir/fungut_forward_reads.qza \
    --o-visualization $data_dir/01/demux_summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/01/demux_summary.qzv


In [3]:
Visualization.load(f"{data_dir}/01/demux_summary.qzv")

<visualization: Visualization uuid: 1c9424b8-2547-4b5e-b7b8-2ae8193ec6b1>

<div style="background-color: yellow; padding: 20px;">
Analysing our demultiplexed sequences using the demux summarize command, we gather the following information:
</div>

- Our sequences contain a minimum of 20567 foreward reads (FR) per sample
- The mean number of FR is 64221.42 
- The maximum of FR is 113461
- Quality score for the FR is stable for the entire sequencing length with the median quality score for position 151 (maximum length) being 38 and few medians dipping only slightly below that.


## 01.02 Metadata editing

Having downloaded our metadata file in the previous chapter, we will now need to edit it slightly to ensure we don't run into any errors in the downstream processing. First, we will replace all numeric values that currently are strings of "Not provided" into Nans so we can work with these numbers even though we don't have a set numeric value. For the categorical variables, we want to keep "Not provided" so that we don't loose them in our diversity analysis and differential abundance analysis. Once we have cleaned the "Not provided" strings, we will upload the new metadata file.

In a second step after cleaning the file, we want to add a column for the rural/urban score of each sample based on the coordinates that are already present in the metadata. We do this by downloading our cleaned file, using QGIS and the RUCC (Rural Urban Conrinuum Codes) and import the edited file with the new column that includes the rural/urban score.

Once all that is done, we will also add a new column to group ranges of BMI to their corresponding category on the BMI scale as a last modification of our metadata before we can get to work.

And with that, we have our finalised metadata file (for now...).

In [ ]:
metadata_clean = pd.read_csv(f"{data_dir}/fungut_metadata.tsv", sep="\t")

In [ ]:
numeric_cols = ["latitude_sample", "longitude_sample", "age_years_sample", "height_cm_sample", "weight_kg_sample", "bmi_sample",] 

for col in numeric_cols:
    if col in metadata_clean.columns:
        metadata_clean[col] = metadata_neu[col].replace(
            ["Not provided"],
            np.nan
        )

In [ ]:
metadata_clean.to_csv("metadata_clean", sep="\t", index=False)

Now we put it into qgis and we add a column for the rural/urban score.

In [ ]:
wget abelad vom rural file "metadata_rucc"

In [ ]:
metadata_rucc = pd.read_csv(f"{data_dir}/metadata_rucc.tsv", sep="\t")

In [ ]:
bins_bmi = [0, 18.5, 24.9, 29.9, 34.9, 39.9, float('inf')]

In [ ]:
labels_bmi = ['Underweight', 'Normal', 'Overweight', 'Adipose 1', 'Adipose 2', 'Adipose 3']

In [ ]:
metadata_ed['BMI_category'] = pd.cut(metadata_rucc'bmi_sample'], bins=bins_bmi, labels=labels_bmi, right=True)

In [ ]:
metadata_ed.to_csv("metadata_ed", sep="\t", index=False)

## 01.03 Metadata exploration

Having freshly cleaned and modified our metadata file, let's have a look at what is actually inside and find out more about where our samples come from.

In [ ]:
graphe vo livia

# 02 Trimming the primers ✂️

In this chapter we remove the primers from the sequences and then create a new summary table to check if we see any impact on the sequence length distribution (Spoiler alert: We don't).

In [4]:
! qiime cutadapt trim-single \
  --i-demultiplexed-sequences $data_dir/fungut_forward_reads.qza \
  --p-front CTTGGTCATTTAGAGGAAGTAA \
  --o-trimmed-sequences $data_dir/02/fungut_forward_reads_trimmed.qza \
  --verbose

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt -u 0 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 1 -q 0,0 --quality-base 33 --cores 1 -o /tmp/qiime2/jovyan/processes/744-1764094159.13@jovyan/tmp/q2-OutPath-jf831hs6/ERR5327198_01_L001_R1_001.fastq.gz --front CTTGGTCATTTAGAGGAAGTAA /tmp/qiime2/jovyan/data/3638611d-1767-413b-9390-70ee3d78e4ff/data/ERR5327198_01_L001_R1_001.fastq.gz

This is cutadapt 5.1 with Python 3.10.14
Command line parameters: -u 0 --

In [5]:
! qiime demux summarize \
  --i-data $data_dir/02/fungut_forward_reads_trimmed.qza \
  --o-visualization $data_dir/02/demux_summary_posttrimming.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/02/demux_summary_posttrimming.qzv


In [6]:
Visualization.load(f"{data_dir}/02/demux_summary_posttrimming.qzv")

<visualization: Visualization uuid: feebbaaf-8aea-4352-bb0a-a19eed7422d4>

<div style="background-color: yellow; padding: 20px;">
Trimming our primers we can see that only XXXXXXXXX of feautres have been edited. Hence, we assume that the primers have already been trimmed as otherwise we would expect many more feautres to have been adjusted.
</div>



# 03 Denoising 🔊 

Ok, now we're all set to actually do some real work with our demultiplexed and trimmed sequences. We start by denoising the sequences to recieve the holy feature table and the denoised sequences that will be the basis of all downstream processing.

- Weli settings heimer brucht
- Denoising benchmarking vor livia

## 03.01 Denoising using DADA2

<div style="background-color: orange; padding: 20px;">
--p-max-ee 4 — controls the maximum expected errors per read accepted by DADA2’s filtering. 4 is a relatively lenient filter; 1–2 is stricter and will remove more reads. Choose this based on the post-trim quality (look at the demux summary).
<\div>

XXXX why did we use the parameters we used?

In [11]:
"""
#Denoising_1
! qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/02/fungut_forward_reads_trimmed.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 1 \
   --p-max-ee 2 \
    --o-representative-sequences $data_dir/03/denoising_1/dada2_rep_seqs_1.qza \
    --o-table $data_dir/03/denoising_1/dada2_table_1.qza \
    --o-denoising-stats $data_dir/03/denoising_1/dada2_stats_1.qza
"""

'\n! qiime dada2 denoise-single    --i-demultiplexed-seqs $data_dir/02/fungut_forward_reads_trimmed.qza    --p-trim-left 0    --p-trunc-len 0    --p-min-fold-parent-over-abundance 1    --p-max-ee 2     --o-representative-sequences $data_dir/03/denoising_1/dada2_rep_seqs_1.qza     --o-table $data_dir/03/denoising_1/dada2_table_1.qza     --o-denoising-stats $data_dir/03/denoising_1/dada2_stats_1.qza\n'

In [7]:
#Denoising_4

! qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/02/fungut_forward_reads_trimmed.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/03/denoising_4/dada2_rep_seqs_4.qza \
    --o-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --o-denoising-stats $data_dir/03/denoising_4/dada2_stats_4.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/03/denoising_4/dada2_table_4.qza
Saved FeatureData[Sequence] to: data/03/denoising_4/dada2_rep_seqs_4.qza
Saved SampleData[DADA2Stats] to: data/03/denoising_4/dada2_stats_4.qza


## 03.02 Checks after denoising for Settings 1

In [ ]:
"""
# Convert denoising statistics to a QZV (shows per-sample read counts through DADA2 steps)
! qiime metadata tabulate \
    --m-input-file $data_dir/03/denoising_1/dada2_stats_1.qza \
    --o-visualization $data_dir/03/denoising_1/dada2_stats_1.qzv
"""

In [ ]:
"""
# Create interactive table of ASV representative sequences (inspect sequence lengths/content)
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/03/denoising_1/dada2_rep_seqs_1.qza \
    --o-visualization $data_dir/03/denoising_1/dada2_rep_seqs_1.qzv
"""

In [ ]:
"""
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/03/denoising_1/dada2_table_1.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/03/denoising_1/dada2_table_1.qzv
"""

In [ ]:
Visualization.load(f"{data_dir}/03/denoising_1/dada2_stats_1.qzv")

In [ ]:
Visualization.load(f"{data_dir}/03/denoising_1/dada2_rep_seqs_1.qzv")

In [ ]:
Visualization.load(f"{data_dir}/03/denoising_1/dada2_table_1.qzv")

## 03.02 Checks after denoising for Settings 4

In [8]:
# Convert denoising statistics to a QZV (shows per-sample read counts through DADA2 steps)
! qiime metadata tabulate \
    --m-input-file $data_dir/03/denoising_4/dada2_stats_4.qza \
    --o-visualization $data_dir/03/denoising_4/dada2_stats_4.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/03/denoising_4/dada2_stats_4.qzv


In [9]:
# Create interactive table of ASV representative sequences (inspect sequence lengths/content)
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/03/denoising_4/dada2_rep_seqs_4.qza \
    --o-visualization $data_dir/03/denoising_4/dada2_rep_seqs_4.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/03/denoising_4/dada2_rep_seqs_4.qzv


In [10]:
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/03/denoising_4/dada2_table_4.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/03/denoising_4/dada2_table_4.qzv


In [19]:
#Check mit neuen Metadaten ob sich was verändert hat wenn man den FT mit denen macht nach denoising
! qiime feature-table summarize \
    --i-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --m-sample-metadata-file $data_dir/met_cleaned_neu.tsv \
    --o-visualization $data_dir/03/denoising_4/dada2_table_4.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/03/denoising_4/dada2_table_4.qzv


In [11]:
Visualization.load(f"{data_dir}/03/denoising_4/dada2_stats_4.qzv")

<visualization: Visualization uuid: b34bda47-8770-48a6-a1f0-3db7621aa785>

In [12]:
Visualization.load(f"{data_dir}/03/denoising_4/dada2_rep_seqs_4.qzv")

<visualization: Visualization uuid: abd4156b-27d6-4db5-a986-7727cd61d7de>

In [13]:
Visualization.load(f"{data_dir}/03/denoising_4/dada2_table_4.qzv")

<visualization: Visualization uuid: e5733419-8d6a-4f94-8589-8a1714899195>

<div style="background-color: yellow; padding: 20px;">
After denoising, .....
</div>



### 03-Checks to run now (after denoising) — pass/fail criteria to decide whether to proceed to taxonomy

Open the three QZVs (you already load them). For each, check the following items.


B. From dada2_table_1.qzv (feature-table summary):

Look at the sample frequency histogram. Decide a minimum read-depth cutoff for downstream analyses (alpha/beta). The cutoff should retain most samples but filter out failures. Document the cutoff.

Look at number of features per sample: if many samples have very few features, they might be failures.

Check whether any samples were dropped entirely (count=0).

C. From dada2_rep_seqs_1.qzv (rep sequences):

Inspect the length distribution of ASVs. ITS has variable lengths — confirm that distribution is biologically plausible and not dominated by very short sequences (primer dimers) or artifacts.

If you find many very short sequences (<100 nt), consider filtering by minimum sequence length before taxonomic classification.

D. Basic cross-checks:

Ensure sample IDs in dada2_table_1.qzv match your metadata fungut_metadata.tsv. Use qiime metadata tabulate --m-input-file fungut_metadata.tsv (to generate metadata qzv) and visually compare IDs. Mismatched names cause downstream failures.

Compute per-sample fraction unassigned after taxonomy (you’ll do this next) and flag samples with e.g., >50% unassigned.

If these checks are acceptable (reasonable retention, acceptable chimera rate, plausible sequence length distribution, few low-depth samples), you can proceed to taxonomy.

# 04 Taxonomy 🍄‍🟫 

Great! Now that we have a finalised metadata file and a denoised feature table, the last thing we need to make all the cool analyses later on is the taxonomic classification for our samples.

To classify our samples, we used a pretrained classifier (INFOS ÜBERE CLASSIFIER) and ran the script below (SCRIPT IRGENDWO IFEèGE WOMER UF EULER BRUCHT HEI) on Euler to recieve a taxonomic classifiaction of our samples. In this notebook, we will first import the output file of the Euler-job and then we will first have a general look at our classification.

## 04.01 Taxonomy overview 

To have a first look at our taxonomic classification, we create the metadata table and the taxa barplots.

In [14]:
! qiime metadata tabulate \
    --m-input-file $data_dir/04/taxjoblinus.qza \
    --o-visualization $data_dir/04/taxonomylinus.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/taxonomylinus.qzv


In [3]:
Visualization.load(f"{data_dir}/04/taxonomylinus.qzv")

<visualization: Visualization uuid: 6e16b4c1-9fb6-4fe7-bd4d-b7c0b2b82ce9>

In [16]:
! qiime taxa barplot \
    --i-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --i-taxonomy $data_dir/04/taxjoblinus.qza \
    --m-metadata-file $data_dir/metadata_ed.tsv \
    --o-visualization $data_dir/04/taxa_bar_plots_overview.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/taxa_bar_plots_overview.qzv


In [4]:
Visualization.load(f"{data_dir}/04/taxa_bar_plots_overview.qzv")

<visualization: Visualization uuid: 523e9bd7-f94f-4fe0-8472-87772d9a4ed1>

## 04.02 Filtering our data to exclude any non-fungal features

Looking at the taxonomic classification barplots, we see some non-fungal features and some samples with a high degree (>80%) of unclassifiable sequences. Let's have a look at some of these sequences:

| sample ID | BLAST | Reason for deletion |
|-------|-------|-------|
| fed424ac80555019c8816c6efb6f2004   | Fungal endophyte isolate   |  No precise classification   |
| 1ada978c8f3ff08af2668393869257ac    | Rhodotorula mucilaginosa  | Mostly from environmental sources | 
| 201dcc24ff7662a5eb05f31a0d0ca6c0    | no BLAST result found   |      |
| 270002b21dcf309398fe1f945dc902e0    | uncultured fugus   |      |
| 27e5e1e9ffac83de20b1ed2784cecee4    | Rhodotorula mucilaginosa   | Mostly from environmental sources |
| 288c841878718e65bd3f27b674c5b16a    | Mucor racemosus   | Mostly from environmental sources  |
| 31bd8178bf3013d48c62813a535b9e09 | Torulaspora delbrueckii |  Mostly from environmental sources |
| 3e2d3eecabe4355bd15aeab85aebaa83 | Rhodotorula mucilaginosa |  Mostly from environmental sources |
| 478e194c1ec118f79940439cc0c7fd36 | Cladosporium allicinum | plant leaves |
| 4af6fc99c1acf99c1b82d1db3a538c9e | Uncultured fungus | |
| 54e88722f1d30a88eaaf19efc686d249 | Uncultured basidiomycete |  |
| 58de98eb759b336275f22b9a976e97e1 | Rhodotorula mucilaginosa | Mostly from environmental sources |
| 5ad3902302653fe8bb0b938de1d18f68 | Uncultured fungus clone |  |
| 68da1589cb5aa90483bf9ddded795ff5 | Uncultured fungus clone |  |
| 73c2e92790e785ec39b58026ed4010ef | Candida glabrata |k__Fungi;p__Ascomycota;sp__Saccharomycotina;c__Saccharomycetes;o__Saccharomycetales;f__Saccharomycetaceae;g__Candida;s__glabrata
  |
| 818fb915612b44bbbd91b854c738a057 | Fungal endophyte | in plants |
| 97d39fb074a634f6e55e91dd89e69c89 | Blastocystis | parasit |
| a0dc22f3ab283a0afd30e730590fb85c | Uncultured fungus |  |
| a2805006a5012b036fa721982c1324fb | Rhizopus oryzae strain |Mostly from environmental sources|
| afbfe0c693e41ecdd14938a045832793 | Rhodotorula mucilaginosa |Mostly from environmental sources  |
| b2cf5a969f4c7a2f29d49b4216a39244 | Kluyveromyces marxianus | Mostly from environmental sources |
| beb8f130fd78d92d09b9aa1e37fec750 | Escherichia coli | Bacteria |
| c6a89f484b775e3a8a0842ec9d7568e1 | Uncultured fungus |  |
| c84004c0020de7d6ff5bde95677e8688 | Uncultured fungus |  |
| c9da8911059b1b0a4a2980d229a06a69 | Candida albicans  | k__Fungi;p__Ascomycota;sp__Saccharomycotina;c__Saccharomycetes;o__Saccharomycetales;f__Debaryomycetaceae;g__Candida;s__albicans
 |
| d8c7c2823a15d6d0e95a8003cf1cc15b | Rhodotorula mucilaginosa |Mostly from environmental sources  |
| d94d771c3041404138de7df8e52c6cd1 | Aethes rutilana genome assembly | Metazoa |
| ddf643cb96fee9d6ecd44d938b35bd61 | Uncultured fungus  |  |
| eff5b3a232c5c37acde9e8fc390d2c12 | no BLAST result found|  |
| f931ac9a7305cdd99a02c25a624e5bb8 | Pichia kudriavzevii  | Mostly from environmental sources |
| fa340e00a973844b027665f54f7f6fb0 | Rhodotorula mucilaginosa  | Mostly from environmental sources |


As we can see from our randomly sampled unclassified features, almost all of the features stem from environmental sources or are from unclultured fungi. The features belonging to environmental samples and those belonging to plants and parasites are most likely contaminations that could distort downstream analyses. For this reason we decided to filter out any non-fungal features and only keep features that have have been classified to the level of class. This is how we do it:

In [18]:
! qiime taxa filter-table \
    --i-table $data_dir/03/denoising_4/dada2_table_4.qza \
    --i-taxonomy $data_dir/04/taxjoblinus.qza \
    --p-exclude Protista,Eukaryota_kgd_Incertae_sedis,Ichthyosporia,Metazoa,Viridiplantae \
    --p-include p__ \
    --o-filtered-table $data_dir/04/dada2_table_filtered.qza

! qiime taxa filter-seqs \
    --i-sequences $data_dir/03/denoising_4/dada2_rep_seqs_4.qza \
    --i-taxonomy $data_dir/04/taxjoblinus.qza \
    --p-exclude Protista,Eukaryota_kgd_Incertae_sedis,Ichthyosporia,Metazoa,Viridiplantae \
    --p-include p__ \
    --o-filtered-sequences $data_dir/04/dada2_rep_seqs_filtered.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/04/dada2_table_filtered.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Sequence] to: data/04/dada2_rep_seqs_filtered.qza


In [ ]:
! qiime feature-table summarize \
    --i-table $data_dir/04/dada2_table_filtered.qza \
    --m-sample-metadata-file $data_dir/metadata_ed.tsv \
    --o-visualization $data_dir/03/denoising_4/dada2_table_4.qzv

## 04.03 Removing samples high in unclassifiable sequences and high in non-fungi

In our taxonomy barplots, we also saw some samples that are very high (>80%) in unclassified sequences or non-fungal sequences. To filter those out, we adjust our metadata and remove the IDs of the unwanted sequences. Then, we create a new feature table summary to see how many samples we lose in the process of filtering.

New metadata edits

fungut_metadata
met_ed
met_ed_rem

In [12]:
metadata_ed = pd.read_csv(f"{data_dir}/metadata_ed.tsv", sep="\t")

In [5]:
rem = ["ERR5327575", "ERR5327509", "ERR5327351", "ERR5327544", "ERR5327300", "ERR5327338", "ERR5327529", "ERR5327533", "ERR5327364", "ERR5327535"]

In [6]:
metadata_ed_rem = metadata_ed[~metadata_ed["ID"].isin(rem)]

In [ ]:
metadata_ed_rem.to_csv("metadata_ed_rem", sep="\t", index=False)

In [30]:
!qiime feature-table filter-samples \
    --i-table $data_dir/04/dada2_table_filtered.qza \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-filtered-table $data_dir/04/dada2_table_filtered_samples_removed.qza 

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/04/dada2_table_filtered_samples_removed.qza


In [31]:
! qiime feature-table summarize \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --o-visualization $data_dir/04/dada2_table_metadata.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/dada2_table_metadata.qzv


In [2]:
Visualization.load(f"{data_dir}/04/dada2_table_metadata.qzv")

<visualization: Visualization uuid: d00deb48-69f4-449b-a6e4-0229e60384bf>

## 04.04 Taxonomy after filtering

In the feautre table summary from the last visualisation, we see that we have now have 140 samples, 10 less than what we started with. Cool, now let's see what has changed in the taxonomic classification after filtering out the unwanted sequences:

In [33]:
!qiime taxa barplot \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --i-taxonomy $data_dir/04/taxjoblinus.qza \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/04/taxa_barplots_filtered.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/04/taxa_barplots_filtered.qzv


In [5]:
Visualization.load(f"{data_dir}/04/taxa_barplots_filtered.qzv")

<visualization: Visualization uuid: db6ab307-ea90-46b6-9656-4cd0cd227aae>

# 05 The path to an appropriate sequencing depth 📐

To ensure we comapre our samples fairly (or what the pros call "statistically correct"), we need to standerdize sample sizes by randomly selecting a certain number of sequences from each sample. This certain number is the sequencing depth (SD) and in the following steps, we will dtermine the SD based on the output of alpha rarefaction  

## 05.01 Finding an appropriate sequencing depth

First we look at the output we get when we do alpha rarefaction at two smartly selected maximal depths (30'000 and 50'000). There we see that both the shannon value plateaus very quickly (already at 1000 features for all metadata columns) and the observed features also pleateau early for most metadata columns. Comparting with how many samples we loose (we don't loose any until around 20'000), we decided on a sequencing depth of 20'000 as all curves have plateaued and we have not lost any samples yet.

In [ ]:
"""
#30000
#! qiime diversity alpha-rarefaction \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza 
    --p-max-depth 30000 \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/05/fungut_alpha_rarefaction_30000.qzv
"""

In [ ]:
"""
#50000
#! qiime diversity alpha-rarefaction \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza 
    --p-max-depth 50000 \
    --m-metadata-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/05/fungut_alpha_rarefaction_50000.qzv
"""

In [ ]:
Visualization.load(f"{data_dir}/05/fungut_alpha_rarefaction_30000.qzv")

In [ ]:
Visualization.load(f"{data_dir}/05/fungut_alpha_rarefaction_50000.qzv")

In the following steps, we check how many samples we would lose fore three possible candidates of sequencing depths by filtering our filtered feature table (so much filtering going on eh?) and summarizing the output to see how many samples we retain.

##### Sequencing depths to test (how many samples do we loose?):
    1. 17500
    2. 20000
    3. 22500
    
Results:
17500: no samples lost (150)
20000: 1 sample lost (149)
22500: 3 samples lost (147)

-> Lets go with 20000. This way we have a good plateau for all curves and lose only 1 sample. 
-> We don't have any extremly low-depth samples that might need filtering (yay!)

In [ ]:
"""
#Test = 17500
!qiime feature-table filter-samples \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --p-min-frequency 17500 \
  --o-filtered-table $data_dir/05/table_minfreq_17500.qza

!qiime feature-table summarize \
  --i-table $data_dir/05/table_minfreq_17500.qza \
  --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/05/table_minfreq_17500_summary.qzv
"""

In [ ]:
"""
#Depth 20000
!qiime feature-table filter-samples \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --p-min-frequency 20000 \
  --o-filtered-table $data_dir/05/table_minfreq_20000.qza

!qiime feature-table summarize \
  --i-table $data_dir/06/table_minfreq_20000.qza \
  --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/table_minfreq_20000_summary.qzv
"""

In [ ]:
"""
#Depth 22500
!qiime feature-table filter-samples \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --p-min-frequency 22500 \
  --o-filtered-table $data_dir/05/table_minfreq_22500.qza

!qiime feature-table summarize \
  --i-table $data_dir/05/table_minfreq_22500.qza \
  --m-sample-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/05/table_minfreq_22500_summary.qzv
"""

In [ ]:
Visualization.load(f"{data_dir}/05/table_minfreq_17500_summary.qzv")

In [ ]:
Visualization.load(f"{data_dir}/05/table_minfreq_20000_summary.qzv")

In [ ]:
Visualization.load(f"{data_dir}/06/table_minfreq_22500_summary.qzv")

## 05.02 Final check before Bootrstrapping

In [ ]:
"""
!qiime diversity alpha-rarefaction \
  --i-table $data_dir/04/dada2_table_filtered.qza \
  --p-max-depth 23000 \
  --p-iterations 50 \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/05/alpha_rarefaction_sanitycheck.qzv
"""

In [36]:
Visualization.load(f"{data_dir}/05/alpha_rarefaction_sanitycheck.qzv")

<visualization: Visualization uuid: 1a970570-a512-4b1e-be7f-4e3b6c6e73d3>

# 06 Diversity analysis 📊

Now that we have our sequencing depth, we can continue analyzing our data and perform diversity analysis. To do so, we investigate both alpha diversity (=within-sample diversity) and beta diversity (=between sample diversity). 

For the actual computation, we rely on k-mer–based diversity metrics. Recent puplications on ITS workflows show that k-mer methods outperform feature-table approaches by avoiding biases introduced during denoising and taxonomy assignment. In practice, this means we compute all diversity metrics directly from sequence-derived k-mers using qiime boots kmer-diversity. We perfor

## 06.01 Kmerizing our ASVs

In [12]:
!qiime boots kmer-diversity ?

Usage: qiime boots kmer-diversity [OPTIONS]

  Given a single feature table as input, this action resamples the feature
  table `n` times to a total frequency of `sampling depth` per sample. It then
  splits all input sequences into kmers, and computes common alpha and beta
  diversity on each resulting kmer table. The resulting artifacts are then
  averaged using the method specified by `alpha_average_method` and
  `beta_average_method` parameters. The resulting average alpha and beta
  diversity artifacts are returned, along with a scatter plot integrated all
  alpha diversity metrics and the PCoA axes for all beta diversity metrics.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency | RelativeFrequency |
    PresenceAbsence]      The input feature table.                  [required]
  --i-sequences ARTIFACT FeatureData[Sequence | RNASequence |
    ProteinSequence]      Input sequences for kmerization.          [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)  

In [11]:
!qiime boots kmer-diversity \
  --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
  --i-sequences $data_dir/04/dada2_rep_seqs_filtered.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --p-sampling-depth 20000 \
  --p-n 1000 \
  --p-replacement \
  --p-kmer-size 16 \
  --p-alpha-average-method median \
  --p-beta-average-method medoid \
  --p-color-by "country_sample" \
  --output-dir $data_dir/06/boots_kmer_diversity_1000

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
^C

Aborted!


<div style="background-color: orange; padding: 20px;">
    ### Questions regarding boots kmer diversity 
- when to use mean when median?
- replace or not replace?
</div>

In [51]:
!qiime metadata tabulate \
    --m-input-file $data_dir/metadata_ed_rem.tsv \
    --o-visualization $data_dir/06/metadata_table.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/metadata_table.qzv


In [52]:
Visualization.load(f"{data_dir}/06/metadata_table.qzv")

<visualization: Visualization uuid: dbae036f-d123-42be-98b7-3ed6aa8d5a81>

## 06.02 Alpha diversity

### 06.02.01 Making the plots

#### Categorical Variables

In [44]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/shannon.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_shannon_group_significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/observed_features.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_observed_features_group_significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/pielou_e.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_pielou_e_group_significance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_shannon_group_significance.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_observed_features_group_significance.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_reso

In [56]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_shannon_group_significance.qzv")

<visualization: Visualization uuid: 7271f8bb-5777-4a42-9e68-b7e850fae714>

In [57]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_observed_features_group_significance.qzv")

<visualization: Visualization uuid: 1910c8b7-d48f-4883-9de0-dfbf009719fb>

In [58]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_pielou_e_group_significance.qzv")

<visualization: Visualization uuid: 3eafc788-2c7a-4583-9950-129161960f32>

#### Numeric Variables

In [45]:
!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/shannon.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_shannon_corrleation.qzv

!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/observed_features.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_observed_features_correlation.qzv

!qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/06/boots-kmer-diversity/alpha_diversities/pielou_e.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/alpha_diversity/alpha_pielou_e_correlation.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_shannon_corrleation.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/alpha_diversity/alpha_observed_features_correlation.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. Th

In [ ]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_shannon_corrleation.qzv")

In [ ]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_observed_features_correlation.qzv")

In [ ]:
Visualization.load(f"{data_dir}/06/alpha_diversity/alpha_pielou_e_correlation.qzv")

### 06.02.2 Alpha diversity findings

<div style="background-color: yellow; padding: 20px;">
Analysing our alpha diversity plots for all three metrics of shannon, observed features and pielou we found the following things:
</div>

**Numeric variables:**
- Age_years_sample is relevant only for richnenss (pValue=0.0174 in observed_feature alpha correlation)
- All other numeric variables showed no significance for all three metrics

**Categorical Variables:**
- Statistical differnce found for **diet type**: both shannon and observed features show relevant p-Values (Shannon: 0.0456, observed features: 0.0154) for comaprisons to all other diet types (Vegan sample size n=5)
- Statistical differnce found for **IDB status**: Significant p-Value (0.0425) for shannon metric when comparing participants with diagnosed IDB (n=10) to participants without (n=117).

- No statistically relevant differences for the following variables:
    - Country
    - State
    - Sex
    - Celiac disease
    - BMI
    - Rural/Urban/peri

## 06.03 Beta diversity

XXX To investigate between-sample diversity (=beta diversity) we take the beta diversity outputs from the q2 boots kmer diversity directory and convert them to emperor plots. In the plots we can then search for any clustering of our samples (spoiler alert: we don't find any clear clustering).

### 06.03.01 Making the plots

In [47]:
# Bray–Curtis
!qiime diversity pcoa \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/braycurtis.qza \
  --o-pcoa $data_dir/06/beta_diversity_plots/kmer_bray_pcoa.qza

!qiime emperor plot \
  --i-pcoa $data_dir/06/beta_diversity_plots/kmer_bray_pcoa.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/beta_diversity_plots/kmer_bray_emperor.qzv

# Jaccard
!qiime diversity pcoa \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/jaccard.qza \
  --o-pcoa $data_dir/06/beta_diversity_plots/kmer_jaccard_pcoa.qza

!qiime emperor plot \
  --i-pcoa $data_dir/06/beta_diversity_plots/kmer_jaccard_pcoa.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --o-visualization $data_dir/06/beta_diversity_plots/kmer_jaccard_emperor.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved PCoAResults to: data/06/beta_diversity_plots/kmer_bray_pcoa.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/beta_diversity_plots/kmer_bray_emperor.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources packa

In [2]:
Visualization.load(f"{data_dir}/06/boots-kmer-diversity/kmer_bray_emperor.qzv")

ValueError: data/06/boots-kmer-diversity/kmer_bray_emperor.qzv does not exist.

In [ ]:
Visualization.load(f"{data_dir}/06/boots-kmer-diversity/kmer_jaccard_emperor.qzv")

### 06.03.02 PERMANOVA

XXXX Having found no clear clustering in our beta diversity plots, we now check if we find statistically relevant differences between our samples using PERMANOVA. 

Variables of interest:
- Diet type
- IDB Status
- BMI
- 

#### Diet type

In [62]:
# Pairwise Bray-Curtis
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/braycurtis.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column diet_type_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_bray_diet_pairwise.qzv

# Pairwise Jaccard
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/jaccard.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column diet_type_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_jaccard_diet_pairwise.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_bray_diet_pairwise.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_jaccard_diet_pairwise.qzv


In [11]:
Visualization.load(f"{data_dir}/06/permanova/permanova_bray_diet_pairwise.qzv")

<visualization: Visualization uuid: b295a990-95f2-4b6d-8744-c71532b7725b>

In [ ]:
Visualization.load(f"{data_dir}/06/permanova/permanova_jaccard_diet_pairwise.qzv")

#### IDB Status

In [64]:
# Pairwise Bray-Curtis
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/braycurtis.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column ibd_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_bray_ibd_pairwise.qzv

# Pairwise Jaccard
!qiime diversity beta-group-significance \
  --i-distance-matrix $data_dir/06/boots-kmer-diversity/distance_matrices/jaccard.qza \
  --m-metadata-file $data_dir/metadata_ed_rem.tsv \
  --m-metadata-column ibd_sample \
  --p-method permanova \
  --p-permutations 999 \
  --p-pairwise \
  --o-visualization $data_dir/06/permanova/permanova_jaccard_ibd_pairwise.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_bray_ibd_pairwise.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06/permanova/permanova_jaccard_ibd_pairwise.qzv


In [8]:
Visualization.load(f"{data_dir}/06/permanova/permanova_bray_ibd_pairwise.qzv")

<visualization: Visualization uuid: f4bb6172-8bc7-4891-9561-60d06ac5efa5>

In [66]:
Visualization.load(f"{data_dir}/06/permanova/permanova_jaccard_ibd_pairwise.qzv")

<visualization: Visualization uuid: 458431e1-a390-4420-9939-c4a778d158c3>

#### BMI 

#### Celiac disease

Columns for PERMANOVA:
country_sample
sex_sample
diet_type_sample
BMI_category
urban/rural/peri

### 06.03.03 PERMDISP

# 07 Differential Abundance 🧮

## 07.01 Filtering our features

Before we start with the *actual" abundnace analysis, let's prepare for it by improving the resolution of the downstream analysis. We do so by first filtering our feature table to only keep features that appear at least 25 times and in at least 4 samples. Then, we will collapse our taxonomic classification at the species level (=7) so that we can later check and compare the differential abundance both at the ASV level and at the species level.

In [7]:
# FT before filtering
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --m-sample-metadata-file $data_dir/met_cleaned_neu.tsv \
    --o-visualization $data_dir/test_table_abundance_before_filtering.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/test_table_abundance_before_filtering.qzv


In [16]:
Visualization.load(f"{data_dir}/test_table_abundance_before_filtering.qzv")

<visualization: Visualization uuid: c6550c35-e9a8-462d-813a-24f45306c607>

In [39]:
! qiime feature-table filter-features \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --p-min-frequency 25 \
    --p-min-samples 4 \
    --o-filtered-table $data_dir/07/table_abundance.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/07/table_abundance.qza


In [18]:
! qiime feature-table filter-features \
    --i-table $data_dir/04/dada2_table_filtered_samples_removed.qza \
    --p-min-frequency 20 \
    --p-min-samples 3 \
    --o-filtered-table $data_dir/07/table_abundance_20_3.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/07/table_abundance_20_3.qza


In [17]:
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/07/table_abundance.qza \
    --m-sample-metadata-file $data_dir/met_cleaned_neu.tsv \
    --o-visualization $data_dir/07/table_abundance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/table_abundance.qzv


In [21]:
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/07/table_abundance_20_3.qza \
    --m-sample-metadata-file $data_dir/met_cleaned_neu.tsv \
    --o-visualization $data_dir/07/table_abundance_20_3.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/table_abundance_20_3.qzv


In [20]:
Visualization.load(f"{data_dir}/07/table_abundance_20_3.qzv")

<visualization: Visualization uuid: b0c9653b-da39-4a0b-bad5-f9ba88e3bef3>

In [17]:
Visualization.load(f"{data_dir}/07/table_abundance.qzv")

<visualization: Visualization uuid: ed8fca90-62c5-4170-a3b8-b87fa9dabb35>

In [19]:
! qiime taxa collapse \
    --i-table $data_dir/07/table_abundance.qza \
    --i-taxonomy $data_dir/04/taxjoblinus.qza \
    --p-level 7 \
    --o-collapsed-table $data_dir/07/table_abundance_collapsed.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/07/table_abundance_collapsed.qza


In [20]:
# Summarize feature table (check per-sample read depth, number of features, and overall counts)
! qiime feature-table summarize \
    --i-table $data_dir/07/table_abundance_collapsed.qza \
    --m-sample-metadata-file $data_dir/met_cleaned_neu.tsv \
    --o-visualization $data_dir/07/table_abundance_collapsed.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/table_abundance_collapsed.qzv


In [5]:
Visualization.load(f"{data_dir}/07/table_abundance_collapsed.qzv")

<visualization: Visualization uuid: 1efd3969-bc87-496d-84f9-87f7a36405cd>

## 07.02 Ancom-BC

Great! Let's now run the actual differnetial analysis, we will run Aconm-BC on the metadata variable of interest on both the ASV level and the species level.

#### Diet type

First for ASV level then also on species level (level 7)

In [4]:
#ASV LEVEL:
# Run ANCOM-BC
! qiime composition ancombc \
    --i-table $data_dir/07/table_abundance.qza \
    --m-metadata-file $data_dir/met_cleaned_neu.tsv \
    --p-formula diet_type_sample \
    --o-differentials $data_dir/07/ancombc/ancombc_diet_differentials.qza

# Generate a barplot of differentially abundant taxa between environments
! qiime composition da-barplot \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials.qza \
    --o-visualization $data_dir/07/ancombcancombc_diet_da_barplot.qzv

# Generate a table of these same values for all taxa
! qiime composition tabulate \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials.qza \
    --o-visualization $data_dir/07/ancombcancombc_diet_results.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: data/07/ancombc/ancombc_diet_differentials.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/ancombcancombc_diet_da_barplot.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_r

In [10]:
#Species LEVEL:
# Run ANCOM-BC
! qiime composition ancombc \
    --i-table $data_dir/07/table_abundance_collapsed.qza \
    --m-metadata-file $data_dir/met_cleaned_neu.tsv \
    --p-formula diet_type_sample \
    --o-differentials $data_dir/07/ancombc/ancombc_diet_differentials_species.qza

# Generate a barplot of differentially abundant taxa between environments
! qiime composition da-barplot \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials_species.qza \
    --o-visualization $data_dir/07/ancombcancombc_diet_da_barplot_species.qzv

# Generate a table of these same values for all taxa
! qiime composition tabulate \
    --i-data $data_dir/07/ancombc/ancombc_diet_differentials_species.qza \
    --o-visualization $data_dir/07/ancombcancombc_diet_results_species.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[DifferentialAbundance] to: data/07/ancombc/ancombc_diet_differentials_species.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/07/ancombcancombc_diet_da_barplot_species.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources

##### ASV Level:

In [10]:
Visualization.load(f"{data_dir}/07/ancombcancombc_diet_da_barplot.qzv")

<visualization: Visualization uuid: 8a5a20fd-e0e0-4e46-97de-1f110b106c0d>

In [9]:
Visualization.load(f"{data_dir}/07/ancombcancombc_diet_results.qzv")

<visualization: Visualization uuid: 74aaaf51-531f-4e85-9d66-bf13be8180e2>

##### Species Level:

In [3]:
Visualization.load(f"{data_dir}/07/ancombcancombc_diet_da_barplot_species.qzv")

<visualization: Visualization uuid: 060ef7df-db04-4939-ac94-9156e6bb147d>

In [14]:
Visualization.load(f"{data_dir}/07/ancombcancombc_diet_results_species.qzv")

<visualization: Visualization uuid: 208b78c8-321b-49c6-ae13-bfbc18e27a7a>

In [ ]:
!qiime composition ancombc2 ?

In [11]:
!qiime composition ancombc2-visualizer ?

Usage: qiime composition ancombc2-visualizer [OPTIONS]

  Displays ANCOMBC2 Log-Fold Change values in a barplot and allows filtering
  based on p-value and standard error. If a taxonomy is provided, features can
  be filtered by taxonomy using an interactive tree.

Inputs:
  --i-data ARTIFACT FeatureData[ANCOMBC2Output]
                         The ANCOMBC2 output to visualize.          [required]
  --i-taxonomy ARTIFACT FeatureData[Taxonomy]
                         The taxonomy associated with the features present in
                         the ANCOMBC2 data.                         [optional]
Outputs:
  --o-visualization VISUALIZATION
                                                                    [required]
Miscellaneous:
  --output-dir PATH      Output unspecified results to a directory
  --verbose / --quiet    Display verbose output to stdout and/or stderr
                         during execution of this action. Or silence output if
                         execution is suc

# XX Machine learning (?)

Withi

Questions for the homie (chatgpt):

In [ ]:
- Does it make sense to run permanova on variables that show no difference in alpha diversity?